In [1]:
import requests
import pandas as pd
from config import census_key
import json




In [2]:
# Creates dataframe of school districts, states, and census codes
name = ['New York City Department of Education',
        'Chicago Public School District',
        'Los Angeles Unified School District',
        'Houston Independent School District',
        'Cave Creek Unified District',
        'Deer Valley Unified District',
        'Glendale Union High School District',
        'Phoenix Union High School District',
        'Tempe Union High School District',
       'Tolleson Union High School District',
        'Paradise Valley Unified District',
        'Scottsdale Unified District',
       'Dallas Independent School District',
       'San Antonio Independent School District',
       'Philadelphia City School District',
        'San Diego City Unified School District',
        'San Jose Unified School District'
       ]
state = ['36','17','06',
         '48','04','04',
         '04','04','04',
         '04','04','04',
         '48','48','42',
         '06','06'
        ]
# The value for Cave Creek Unified District is incorrect due to unexpected error. Do not use values returned for Cave Creek
district = ['20580','09930','22710',
            '23640','07750','07750',
            '03450','06330','08340',
            '08520','05930','07570',
            '16230','38730','18990',
            '34320', '34590'
           ]

full_code = ['9700000US3620580','9700000US1709930','9700000US0622710',
             '9700000US4823640','9700000US0400001','9700000US0407750',
             '9600000US0403450','9600000US0406330', '9600000US0408340',
             '9600000US0408520','9700000US0405930', '9700000US0407570',
             '9700000US4816230','9700000US4838730','9700000US4218990',
             '9700000US0634320','9700000US0634590'
            ]
districts_df = pd.DataFrame(name)
districts_df['State'] = state
districts_df['District'] = district
districts_df['Full_Code'] = full_code
districts_df.columns = ['Name', 'State', 'District', 'Full_Code']
# Unified and secondary need slightly different API requests
districts_df['Unified_Secondary'] = ['U','U','U',
                                     'U','U','U',
                                     'S','S','S',
                                     'S','U','U',
                                    'U','U','U',
                                    'U','U']
districts_df17 = districts_df
districts_df18 = districts_df

### 2018 data

In [3]:
# API call loop for table B14005 2018
# some setup before the loop
#set base url for 2018
base_url = 'https://api.census.gov/data/2018/acs/acs1?'
group = "B14005"
district = ""
state = ""
selectB14005_full = []
# loop through each district in districts_df
for index, row in districts_df18.iterrows():
    district = row['District']
    state = row['State']
    u_s = row['Unified_Secondary']
    # Check for unified
    if u_s == 'U':
        try:
            print(f"Trying {row['Name']} with {district} and {state}")
            response = requests.get(f"{base_url}get=group({group})&for=school%20district%20(unified):{district}&in=state:{state}&key={census_key}").json()
        except:
            print(f" request failed with : {base_url}get=group({group})&for=school%20district%20(unified):{district}&in=state:{state}&key={census_key}") 
    # Check for secondary
    elif u_s == 'S':
        # Skips secondary districts due to lack of data
        try:
            #print(f"Trying {row['Name']} with {district} and {state}")
            #response = requests.get(f"{base_url}get=group({group})&for=school%20district%20(secondary):{district}&in=state:{state}&key={census_key}").json()
            continue
        except:
            print(f" request failed with : {base_url}get=group({group})&for=school%20district%20(secondary):{district}&in=state:{state}&key={census_key}")
    # Clean call response
    response_df = pd.DataFrame(response)
    response_df.columns = response_df.iloc[0]
    # trim columns
    selectB14005 = pd.DataFrame(response_df.iloc[1][['GEO_ID','state','school district (unified)',
                    'B14005_001E','B14005_002E','B14005_003E','B14005_004E',
                     'B14005_005E','B14005_006E','B14005_016E','B14005_017E',
                    'B14005_018E','B14005_019E', 'B14005_020E']])
    select2B14005 = selectB14005.transpose()
    # add to districts_df18
    selectB14005_full.append(select2B14005.iloc[0])
    
selectB14005_full_df = pd.DataFrame(selectB14005_full)
districts_df18B14005 = districts_df18.merge(selectB14005_full_df, left_on = 'Full_Code', right_on = 'GEO_ID')

Trying New York City Department of Education with 20580 and 36
Trying Chicago Public School District with 09930 and 17
Trying Los Angeles Unified School District with 22710 and 06
Trying Houston Independent School District with 23640 and 48
Trying Cave Creek Unified District with 07750 and 04
Trying Deer Valley Unified District with 07750 and 04
Trying Paradise Valley Unified District with 05930 and 04
Trying Scottsdale Unified District with 07570 and 04
Trying Dallas Independent School District with 16230 and 48
Trying San Antonio Independent School District with 38730 and 48
Trying Philadelphia City School District with 18990 and 42
Trying San Diego City Unified School District with 34320 and 06
Trying San Jose Unified School District with 34590 and 06


In [4]:
# replace empty cells with zero
districts_df18B14005 = districts_df18B14005.fillna(0)
# Rename B14005_001E to Tot. It is the total 16 to 19 y/os
districts_df18B14005['Tot'] = districts_df18B14005['B14005_001E'].astype('int64') + districts_df18B14005['B14005_017E'].astype('int64')


In [5]:
# Calculate total enrolled
# (male enrolled + female enrolled)
districts_df18B14005['TotEnrll'] = districts_df18B14005['B14005_003E'].astype('int64') + districts_df18B14005['B14005_017E'].astype('int64')

# Calculate total enrolled and employed
# (male enrolled and employed + female enrolled and employed)
districts_df18B14005['TotEnrllEmply'] = districts_df18B14005['B14005_004E'].astype('int64') + districts_df18B14005['B14005_018E'].astype('int64')

# Calculate total enrolled and unemployed
# (male enrolled and unemployed + female enrolled and unemployed)
districts_df18B14005['TotEnrllUnEmply'] = districts_df18B14005['B14005_005E'].astype('int64') + districts_df18B14005['B14005_019E'].astype('int64')

# Calculate total enrolled and not in labor force
# (male enrolled and not in labor force + female enrolled and not in labor force)
districts_df18B14005['TotEnrllNonLbr'] = districts_df18B14005['B14005_006E'].astype('int64') + districts_df18B14005['B14005_020E'].astype('int64')

# Calculate total enrolled and not in labor force
# (male enrolled and not in labor force + female enrolled and not in labor force)
districts_df18B14005['TotEnrllLbr'] = districts_df18B14005['TotEnrll'].astype('int64') - districts_df18B14005['TotEnrllNonLbr'].astype('int64')

In [6]:
# Calculate employment rate for enrolled
# (total enrolled and employed / total enrolled and in labor force)
districts_df18B14005['EmplyRateEnrll'] = districts_df18B14005['TotEnrllEmply'].astype('float64') / districts_df18B14005['TotEnrllLbr'].astype('float64')

# Calculate labor force participation rate for enrolled
# (total enrolled and in labor force / total enrolled)
districts_df18B14005['LbrRateEnrll'] = districts_df18B14005['TotEnrllLbr'].astype('float64') / districts_df18B14005['TotEnrll'].astype('float64')

# Calculate employment rate for enrolled
# (total enrolled and unemployed / total enrolled and in labor force)
districts_df18B14005['UnEmplyRateEnrll'] = districts_df18B14005['TotEnrllUnEmply'].astype('float64') / districts_df18B14005['TotEnrllLbr'].astype('float64')

# Calculate not in labor force for enrolled
# (total enrolled and not in labor force / total enrolled)
districts_df18B14005['NonLbrRateEnrll'] = districts_df18B14005['TotEnrllNonLbr'].astype('float64') / districts_df18B14005['TotEnrll'].astype('float64')


In [7]:
# Create summary table
summary_18BB14005 = districts_df18B14005[['Name','Full_Code','TotEnrll','TotEnrllEmply','EmplyRateEnrll','TotEnrllUnEmply','UnEmplyRateEnrll','TotEnrllLbr','LbrRateEnrll','TotEnrllNonLbr','NonLbrRateEnrll']]
summary_18BB14005.to_csv('output/summary_18B14005.csv')

In [8]:
# API call loop for table B23002A-B23002I 2018 which give employment by age group for ethnic groups
# some setup before the loop
#set base url for 2018
base_url = 'https://api.census.gov/data/2018/acs/acs1?'


group_list = ["B23002B", "B23002C", "B23002D", "B23002E",
              "B23002F", "B23002G", "B23002H", "B23002I"]
district = ""
state = ""
selectB23002_full = []
# loop through each district in districts_df
for index, row in districts_df18.iterrows():
    district = row['District']
    state = row['State']
    u_s = row['Unified_Secondary']
    # loop through each table in group list
    for i in group_list:
        # Check for unified school district
        if u_s == 'U':
            try:
                print(f"Trying {row['Name']} with {district} and {state}")
                response = requests.get(f"{base_url}get=group({i})&for=school%20district%20(unified):{district}&in=state:{state}&key={census_key}").json()
            except:
                print(f" request failed with : {base_url}get=group({i})&for=school%20district%20(unified):{district}&in=state:{state}&key={census_key}") 
        # Check for secondary district
        elif u_s == 'S':
            # Skipping secondary districts. After testing, some of the secondary districts do not have any data for 16-19 age group.
            # The missing columns in the response creates the error below.
            try:
                print(f"Skipping {row['Name']} on {i}")
                #print(f"Trying {row['Name']} with {district} and {state}")
                #response = requests.get(f"{base_url}get=group({i})&for=school%20district%20(secondary):{district}&in=state:{state}&key={census_key}").json()
                continue
            except:
                print(f" request failed with : {base_url}get=group({i})&for=school%20district%20(secondary):{district}&in=state:{state}&key={census_key}")
        # Clean call response
        response_df = pd.DataFrame(response)
        #print(response_df)
        response_df.columns = response_df.iloc[0]
        # trim columns
        # set up column names based on current table from group list
        selectcolumns = ['GEO_ID','state','school district (unified)',
                        i+'_003E',i+'_004E',
                     i+'_005E',i+'_006E',i+'_007E',i+'_008E',
                    i+'_009E',i+'_042E', i+'_043E',i+'_044E',
                    i+'_045E',i+'_046E',i+'_047E',i+'_048E']
        # The next line currently causes an error. "Passing list-likes to .loc or [] with any missing labels is no longer supported"
        # The error is caused by the first Glendale Union High request. Error may be caused by secondary school request.
        # After testing, some secondary districts have no data which causes errors as cleaning is attempted
        selectB23002 = pd.DataFrame(response_df.iloc[1][selectcolumns])
        select2B23002 = selectB23002.transpose()
    
        # add to districts_df18
        selectB23002_full.append(select2B23002.iloc[0])
        selectB23002_full_df = pd.DataFrame(selectB23002_full)
        #print(selectB23002_full_df)
        districts_df18B23002 = districts_df18.merge(selectB23002_full_df, left_on = 'Full_Code', right_on = 'GEO_ID')


Trying New York City Department of Education with 20580 and 36
Trying New York City Department of Education with 20580 and 36
Trying New York City Department of Education with 20580 and 36
Trying New York City Department of Education with 20580 and 36
Trying New York City Department of Education with 20580 and 36
Trying New York City Department of Education with 20580 and 36
Trying New York City Department of Education with 20580 and 36
Trying New York City Department of Education with 20580 and 36
Trying Chicago Public School District with 09930 and 17
Trying Chicago Public School District with 09930 and 17
Trying Chicago Public School District with 09930 and 17
Trying Chicago Public School District with 09930 and 17
Trying Chicago Public School District with 09930 and 17
Trying Chicago Public School District with 09930 and 17
Trying Chicago Public School District with 09930 and 17
Trying Chicago Public School District with 09930 and 17
Trying Los Angeles Unified School District with 

In [9]:
group_list = ["B23002B", "B23002C", "B23002D", "B23002E",
              "B23002F", "B23002G", "B23002H", "B23002I"]
# create dicitionary to convert table name from group_list into abbreviation for ethnic group
# some of these categories are pretty limiting, but it's how the census is organized 
# BLK = African American
# IND = Native American / Native Alaskan
# ASN = Asian
# HIP = Native Hawaiian / Pacific islander
# OTH = Other
# BIR = Two or more races
# WHT = White alone, nonhispanic
# LAT = Hispanic or Latino
group_name = {"B23002B":'BLK', "B23002C":'IND', "B23002D":'ASN', "B23002E":'HIP',
              "B23002F": 'OTH', "B23002G":'BIR', "B23002H":'WHT', "B23002I":'LAT'}
# group by district code to remove 'NaN' cells
districts_df18B23002_grouped = districts_df18B23002.groupby('Full_Code').first().reset_index()
# replace empty cells with zero
districts_df18B23002_grouped = districts_df18B23002_grouped.fillna(0)
#create list of all columns going into summary file
summary_columns = ['Name','Full_Code','Unified_Secondary']

# find total 16-19 y/os per ethnic group (male + female for each group)
for i in group_list:
    # total males 16-19 y/os estimate
    col1name = i + '_003E'
    # total females 16-19 y/os estimate
    col2name = i + '_042E'
    # creates column name. 'Tot' stands for total
    new_colname = group_name[i] + 'Tot'
    # add males to females. Census only includes these two categories
    sumcol = districts_df18B23002_grouped[col1name].astype('int64') + districts_df18B23002_grouped[col2name].astype('int64')
    districts_df18B23002_grouped[new_colname] = sumcol
    # add calculated column to list of columns for summary dataframe
    summary_columns.append(new_colname)
    
#find 16-19 y/os in military per ethnic group. Needed for labor force participation calc
# (male in armed forces + female in armed forces)
for i in group_list:
    # find total 16-19 y/os in armed forces
    col2name = i + '_005E'
    col3name = i + '_044E'
    tot_mil = districts_df18B23002_grouped[col2name] + districts_df18B23002_grouped[col3name]
    # creates column name. 'Mil' stands for military
    new_colname = group_name[i] + 'Mil'
    new_col = districts_df18B23002_grouped[col2name].astype('int64') + districts_df18B23002_grouped[col3name].astype('int64')
    districts_df18B23002_grouped[new_colname] = new_col
    # add calculated column to list of columns for summary dataframe
    summary_columns.append(new_colname)
    
# find 16-19 y/os not in military per ethnic group. Needed for labor force participation calc 
# (total - military)
for i in group_list:
    # find total 16-19 y/os in armed forces
    col1name = group_name[i] + 'Tot'
    col2name = group_name[i] + 'Mil'
    # creates column name. 'Civ' stands for civilian
    new_colname = group_name[i] + 'Civ'
    # Subtract military estimate from total population
    new_col = districts_df18B23002_grouped[col1name].astype('int64') - districts_df18B23002_grouped[col2name].astype('int64')
    districts_df18B23002_grouped[new_colname] = new_col
    # add calculated column to list of columns for summary dataframe
    summary_columns.append(new_colname)

In [10]:
# find total 16-19 y/os in civilian labor force
# (male in labor force + female in labor force)
for i in group_list:
    # total males 16-19 y/os in civilian labor force estimate
    col1name = i + '_006E'
    # total females 16-19 y/os in civilian labor force estimate
    col2name = i + '_045E'
    # creates column name. 'CivLbr' stands for civilian labor force
    new_colname = group_name[i] + 'CivLbr'
    # add males to females. Census only includes these two categories
    newcol = districts_df18B23002_grouped[col1name].astype('int64') + districts_df18B23002_grouped[col2name].astype('int64')
    districts_df18B23002_grouped[new_colname] = newcol
    # add calculated column to list of columns for summary dataframe
    summary_columns.append(new_colname)

In [11]:
# find civilian labor force participation rate
# (Civilian Labor Force / total civilian)
for i in group_list:
    # total civilian labor force estimate
    col1name = group_name[i] + 'CivLbr'
    # total civilians
    col2name =group_name[i] + 'Civ'
    # creates column name. 'CivLbr' stands for civilian labor force participation rate
    new_colname = group_name[i] + 'CivLbrRate'
    # divid labor force estimate by total civilians
    newcol = districts_df18B23002_grouped[col1name].astype('float64') / districts_df18B23002_grouped[col2name].astype('float64')
    districts_df18B23002_grouped[new_colname] = newcol
    # add calculated column to list of columns for summary dataframe
    summary_columns.append(new_colname)

In [12]:
# find count and percent of population outside the labor force
# out of labor force count
# (civilian male out of labor force + civilian female out of labor force)
for i in group_list:
    # total male civilian unemployment estimate
    col1name = i + '_009E'
    # total female civilian unemployment estimate
    col2name =i + '_048E'
    # creates column name. 'TotNonLbr' stands for total civilians not in labor force
    new_colname = group_name[i] + 'TotNonLbr'
    # add males to females. Census only includes these two categories
    newcol = districts_df18B23002_grouped[col1name].astype('int64') + districts_df18B23002_grouped[col2name].astype('int64')
    districts_df18B23002_grouped[new_colname] = newcol
    # add calculated column to list of columns for summary dataframe
    summary_columns.append(new_colname)
    
# find percent of civilians out of labor force
# (Civilian not in Labor Force / total civilian)
for i in group_list:
    # total civilian not in labor force estimate
    col1name = group_name[i] + 'TotNonLbr'
    # total civilians
    col2name =group_name[i] + 'Civ'
    # creates column name. 'CivLbr' stands for civilian labor force participation rate
    new_colname = group_name[i] + 'TotNonLbrRate'
    # divid labor force estimate by total civilians
    newcol = districts_df18B23002_grouped[col1name].astype('float64') / districts_df18B23002_grouped[col2name].astype('float64')
    districts_df18B23002_grouped[new_colname] = newcol
    # add calculated column to list of columns for summary dataframe
    summary_columns.append(new_colname)


In [13]:
# find civilian employed and unemployed counts

# employement count
# (civilian male employed + civilian female employed)
for i in group_list:
    # total male civilian employment estimate
    col1name = i + '_007E'
    # total female civilian employment estimate
    col2name =i + '_046E'
    # creates column name. 'TotEmply' stands for total civilians employed
    new_colname = group_name[i] + 'TotEmply'
    # add males to females. Census only includes these two categories
    newcol = districts_df18B23002_grouped[col1name].astype('int64') + districts_df18B23002_grouped[col2name].astype('int64')
    districts_df18B23002_grouped[new_colname] = newcol
    # add calculated column to list of columns for summary dataframe
    summary_columns.append(new_colname)

# unemployement count
# (civilian male unemployed + civilian female unemployed)
for i in group_list:
    # total male civilian unemployment estimate
    col1name = i + '_008E'
    # total female civilian unemployment estimate
    col2name =i + '_047E'
    # creates column name. 'TotEmply' stands for total civilians unemployed
    new_colname = group_name[i] + 'TotUnEmply'
    # add males to females. Census only includes these two categories
    newcol = districts_df18B23002_grouped[col1name].astype('int64') + districts_df18B23002_grouped[col2name].astype('int64')
    districts_df18B23002_grouped[new_colname] = newcol
    # add calculated column to list of columns for summary dataframe
    summary_columns.append(new_colname)
    


In [14]:
# find civilian employed and unemployed rates

# employement rate
# (total civilians employed / total civilian labor force)
for i in group_list:
    # total civilian employment estimate
    col1name = group_name[i] + 'TotEmply'
    # total civilian labor force
    col2name = group_name[i] + 'CivLbr'
    # creates column name. 'EmplyRate' stands for employment rate
    new_colname = group_name[i] + 'EmplyRate'
    # divide employment estimate by total labor force
    newcol = districts_df18B23002_grouped[col1name].astype('float64') / districts_df18B23002_grouped[col2name].astype('float64')
    districts_df18B23002_grouped[new_colname] = newcol
    # add calculated column to list of columns for summary dataframe
    summary_columns.append(new_colname)

# unemployement rate
# (total civilians unemployed / total civilian labor force)
for i in group_list:
    # total civilian unemployment estimate
    col1name = group_name[i] + 'TotUnEmply'
    # total civilian labor force
    col2name = group_name[i] + 'CivLbr'
    # creates column name. 'UnEmplyRate' stands for unemployment rate
    new_colname = group_name[i] + 'UnEmplyRate'
    # divide unemployment estimate by total labor force
    newcol = districts_df18B23002_grouped[col1name].astype('float64') / districts_df18B23002_grouped[col2name].astype('float64')
    districts_df18B23002_grouped[new_colname] = newcol
    # add calculated column to list of columns for summary dataframe
    summary_columns.append(new_colname)

In [15]:
summary_18B23002 = districts_df18B23002_grouped[summary_columns]
summary_18B23002.to_csv('output/summary_18B23002.csv')

### 2017 data

In [16]:
# API call loop for table B14005 2017
# some setup before the loop
#set base url for 2017
base_url = 'https://api.census.gov/data/2017/acs/acs1?'
group = "B14005"
district = ""
state = ""
selectB14005_full = []
# loop through each district in districts_df
for index, row in districts_df17.iterrows():
    district = row['District']
    state = row['State']
    u_s = row['Unified_Secondary']
    # Check for unified
    if u_s == 'U':
        try:
            print(f"Trying {row['Name']} with {district} and {state}")
            response = requests.get(f"{base_url}get=group({group})&for=school%20district%20(unified):{district}&in=state:{state}&key={census_key}").json()
        except:
            print(f" request failed with : {base_url}get=group({group})&for=school%20district%20(unified):{district}&in=state:{state}&key={census_key}") 
    # Check for secondary
    elif u_s == 'S':
        # Skips secondary districts due to lack of data
        try:
            #print(f"Trying {row['Name']} with {district} and {state}")
            #response = requests.get(f"{base_url}get=group({group})&for=school%20district%20(secondary):{district}&in=state:{state}&key={census_key}").json()
            continue
        except:
            print(f" request failed with : {base_url}get=group({group})&for=school%20district%20(secondary):{district}&in=state:{state}&key={census_key}")
    # Clean call response
    response_df = pd.DataFrame(response)
    response_df.columns = response_df.iloc[0]
    # trim columns
    selectB14005 = pd.DataFrame(response_df.iloc[1][['GEO_ID','state','school district (unified)',
                    'B14005_001E','B14005_002E','B14005_003E','B14005_004E',
                     'B14005_005E','B14005_006E','B14005_016E','B14005_017E',
                    'B14005_018E','B14005_019E', 'B14005_020E']])
    select2B14005 = selectB14005.transpose()
    # add to districts_df17
    selectB14005_full.append(select2B14005.iloc[0])
    
selectB14005_full_df = pd.DataFrame(selectB14005_full)
districts_df17B14005 = districts_df17.merge(selectB14005_full_df, left_on = 'Full_Code', right_on = 'GEO_ID')

Trying New York City Department of Education with 20580 and 36
Trying Chicago Public School District with 09930 and 17
Trying Los Angeles Unified School District with 22710 and 06
Trying Houston Independent School District with 23640 and 48
Trying Cave Creek Unified District with 07750 and 04
Trying Deer Valley Unified District with 07750 and 04
Trying Paradise Valley Unified District with 05930 and 04
Trying Scottsdale Unified District with 07570 and 04
Trying Dallas Independent School District with 16230 and 48
Trying San Antonio Independent School District with 38730 and 48
Trying Philadelphia City School District with 18990 and 42
Trying San Diego City Unified School District with 34320 and 06
Trying San Jose Unified School District with 34590 and 06


In [17]:
# replace empty cells with zero
districts_df17B14005 = districts_df17B14005.fillna(0)
# Rename B14005_001E to Tot. It is the total 16 to 19 y/os
districts_df17B14005['Tot'] = districts_df17B14005['B14005_001E'].astype('int64') + districts_df17B14005['B14005_017E'].astype('int64')


In [18]:
# Calculate total enrolled
# (male enrolled + female enrolled)
districts_df17B14005['TotEnrll'] = districts_df17B14005['B14005_003E'].astype('int64') + districts_df17B14005['B14005_017E'].astype('int64')

# Calculate total enrolled and employed
# (male enrolled and employed + female enrolled and employed)
districts_df17B14005['TotEnrllEmply'] = districts_df17B14005['B14005_004E'].astype('int64') + districts_df17B14005['B14005_018E'].astype('int64')

# Calculate total enrolled and unemployed
# (male enrolled and unemployed + female enrolled and unemployed)
districts_df17B14005['TotEnrllUnEmply'] = districts_df17B14005['B14005_005E'].astype('int64') + districts_df17B14005['B14005_019E'].astype('int64')

# Calculate total enrolled and not in labor force
# (male enrolled and not in labor force + female enrolled and not in labor force)
districts_df17B14005['TotEnrllNonLbr'] = districts_df17B14005['B14005_006E'].astype('int64') + districts_df17B14005['B14005_020E'].astype('int64')

# Calculate total enrolled and not in labor force
# (male enrolled and not in labor force + female enrolled and not in labor force)
districts_df17B14005['TotEnrllLbr'] = districts_df17B14005['TotEnrll'].astype('int64') - districts_df17B14005['TotEnrllNonLbr'].astype('int64')

In [19]:
# Calculate employment rate for enrolled
# (total enrolled and employed / total enrolled and in labor force)
districts_df17B14005['EmplyRateEnrll'] = districts_df17B14005['TotEnrllEmply'].astype('float64') / districts_df17B14005['TotEnrllLbr'].astype('float64')

# Calculate labor force participation rate for enrolled
# (total enrolled and in labor force / total enrolled)
districts_df17B14005['LbrRateEnrll'] = districts_df17B14005['TotEnrllLbr'].astype('float64') / districts_df17B14005['TotEnrll'].astype('float64')

# Calculate employment rate for enrolled
# (total enrolled and unemployed / total enrolled and in labor force)
districts_df17B14005['UnEmplyRateEnrll'] = districts_df17B14005['TotEnrllUnEmply'].astype('float64') / districts_df17B14005['TotEnrllLbr'].astype('float64')

# Calculate not in labor force for enrolled
# (total enrolled and not in labor force / total enrolled)
districts_df17B14005['NonLbrRateEnrll'] = districts_df17B14005['TotEnrllNonLbr'].astype('float64') / districts_df17B14005['TotEnrll'].astype('float64')


In [20]:
# Create summary table
summary_17BB14005 = districts_df17B14005[['Name','Full_Code','TotEnrll','TotEnrllEmply','EmplyRateEnrll','TotEnrllUnEmply','UnEmplyRateEnrll','TotEnrllLbr','LbrRateEnrll','TotEnrllNonLbr','NonLbrRateEnrll']]
summary_17BB14005.to_csv('output/summary_17B14005.csv')

In [21]:
# API call loop for table B23002A-B23002I 2017 which give employment by age group for ethnic groups
# some setup before the loop
#set base url for 2017
base_url = 'https://api.census.gov/data/2017/acs/acs1?'


group_list = ["B23002B", "B23002C", "B23002D", "B23002E",
              "B23002F", "B23002G", "B23002H", "B23002I"]
district = ""
state = ""
selectB23002_full = []
# loop through each district in districts_df
for index, row in districts_df17.iterrows():
    district = row['District']
    state = row['State']
    u_s = row['Unified_Secondary']
    # loop through each table in group list
    for i in group_list:
        # Check for unified school district
        if u_s == 'U':
            try:
                print(f"Trying {row['Name']} with {district} and {state}")
                response = requests.get(f"{base_url}get=group({i})&for=school%20district%20(unified):{district}&in=state:{state}&key={census_key}").json()
            except:
                print(f" request failed with : {base_url}get=group({i})&for=school%20district%20(unified):{district}&in=state:{state}&key={census_key}") 
        # Check for secondary district
        elif u_s == 'S':
            # Skipping secondary districts. After testing, some of the secondary districts do not have any data for 16-19 age group.
            # The missing columns in the response creates the error below.
            try:
                print(f"Skipping {row['Name']} on {i}")
                #print(f"Trying {row['Name']} with {district} and {state}")
                #response = requests.get(f"{base_url}get=group({i})&for=school%20district%20(secondary):{district}&in=state:{state}&key={census_key}").json()
                continue
            except:
                print(f" request failed with : {base_url}get=group({i})&for=school%20district%20(secondary):{district}&in=state:{state}&key={census_key}")
        # Clean call response
        response_df = pd.DataFrame(response)
        #print(response_df)
        response_df.columns = response_df.iloc[0]
        # trim columns
        # set up column names based on current table from group list
        selectcolumns = ['GEO_ID','state','school district (unified)',
                        i+'_003E',i+'_004E',
                     i+'_005E',i+'_006E',i+'_007E',i+'_008E',
                    i+'_009E',i+'_042E', i+'_043E',i+'_044E',
                    i+'_045E',i+'_046E',i+'_047E',i+'_048E']
        # The next line currently causes an error. "Passing list-likes to .loc or [] with any missing labels is no longer supported"
        # The error is caused by the first Glendale Union High request. Error may be caused by secondary school request.
        # After testing, some secondary districts have no data which causes errors as cleaning is attempted
        selectB23002 = pd.DataFrame(response_df.iloc[1][selectcolumns])
        select2B23002 = selectB23002.transpose()
    
        # add to districts_df17
        selectB23002_full.append(select2B23002.iloc[0])
        selectB23002_full_df = pd.DataFrame(selectB23002_full)
        #print(selectB23002_full_df)
        districts_df17B23002 = districts_df17.merge(selectB23002_full_df, left_on = 'Full_Code', right_on = 'GEO_ID')


Trying New York City Department of Education with 20580 and 36
Trying New York City Department of Education with 20580 and 36
Trying New York City Department of Education with 20580 and 36
Trying New York City Department of Education with 20580 and 36
Trying New York City Department of Education with 20580 and 36
Trying New York City Department of Education with 20580 and 36
Trying New York City Department of Education with 20580 and 36
Trying New York City Department of Education with 20580 and 36
Trying Chicago Public School District with 09930 and 17
Trying Chicago Public School District with 09930 and 17
Trying Chicago Public School District with 09930 and 17
Trying Chicago Public School District with 09930 and 17
Trying Chicago Public School District with 09930 and 17
Trying Chicago Public School District with 09930 and 17
Trying Chicago Public School District with 09930 and 17
Trying Chicago Public School District with 09930 and 17
Trying Los Angeles Unified School District with 

In [22]:
group_list = ["B23002B", "B23002C", "B23002D", "B23002E",
              "B23002F", "B23002G", "B23002H", "B23002I"]
# create dicitionary to convert table name from group_list into abbreviation for ethnic group
# some of these categories are pretty limiting, but it's how the census is organized 
# BLK = African American
# IND = Native American / Native Alaskan
# ASN = Asian
# HIP = Native Hawaiian / Pacific islander
# OTH = Other
# BIR = Two or more races
# WHT = White alone, nonhispanic
# LAT = Hispanic or Latino
group_name = {"B23002B":'BLK', "B23002C":'IND', "B23002D":'ASN', "B23002E":'HIP',
              "B23002F": 'OTH', "B23002G":'BIR', "B23002H":'WHT', "B23002I":'LAT'}
# group by district code to remove 'NaN' cells
districts_df17B23002_grouped = districts_df17B23002.groupby('Full_Code').first().reset_index()
# replace empty cells with zero
districts_df17B23002_grouped = districts_df17B23002_grouped.fillna(0)
#create list of all columns going into summary file
summary_columns = ['Name','Full_Code','Unified_Secondary']

# find total 16-19 y/os per ethnic group (male + female for each group)
for i in group_list:
    # total males 16-19 y/os estimate
    col1name = i + '_003E'
    # total females 16-19 y/os estimate
    col2name = i + '_042E'
    # creates column name. 'Tot' stands for total
    new_colname = group_name[i] + 'Tot'
    # add males to females. Census only includes these two categories
    sumcol = districts_df17B23002_grouped[col1name].astype('int64') + districts_df17B23002_grouped[col2name].astype('int64')
    districts_df17B23002_grouped[new_colname] = sumcol
    # add calculated column to list of columns for summary dataframe
    summary_columns.append(new_colname)
    
#find 16-19 y/os in military per ethnic group. Needed for labor force participation calc
# (male in armed forces + female in armed forces)
for i in group_list:
    # find total 16-19 y/os in armed forces
    col2name = i + '_005E'
    col3name = i + '_044E'
    tot_mil = districts_df17B23002_grouped[col2name] + districts_df17B23002_grouped[col3name]
    # creates column name. 'Mil' stands for military
    new_colname = group_name[i] + 'Mil'
    new_col = districts_df17B23002_grouped[col2name].astype('int64') + districts_df17B23002_grouped[col3name].astype('int64')
    districts_df17B23002_grouped[new_colname] = new_col
    # add calculated column to list of columns for summary dataframe
    summary_columns.append(new_colname)
    
# find 16-19 y/os not in military per ethnic group. Needed for labor force participation calc 
# (total - military)
for i in group_list:
    # find total 16-19 y/os in armed forces
    col1name = group_name[i] + 'Tot'
    col2name = group_name[i] + 'Mil'
    # creates column name. 'Civ' stands for civilian
    new_colname = group_name[i] + 'Civ'
    # Subtract military estimate from total population
    new_col = districts_df17B23002_grouped[col1name].astype('int64') - districts_df17B23002_grouped[col2name].astype('int64')
    districts_df17B23002_grouped[new_colname] = new_col
    # add calculated column to list of columns for summary dataframe
    summary_columns.append(new_colname)

In [23]:
# find total 16-19 y/os in civilian labor force
# (male in labor force + female in labor force)
for i in group_list:
    # total males 16-19 y/os in civilian labor force estimate
    col1name = i + '_006E'
    # total females 16-19 y/os in civilian labor force estimate
    col2name = i + '_045E'
    # creates column name. 'CivLbr' stands for civilian labor force
    new_colname = group_name[i] + 'CivLbr'
    # add males to females. Census only includes these two categories
    newcol = districts_df17B23002_grouped[col1name].astype('int64') + districts_df17B23002_grouped[col2name].astype('int64')
    districts_df17B23002_grouped[new_colname] = newcol
    # add calculated column to list of columns for summary dataframe
    summary_columns.append(new_colname)

In [24]:
# find civilian labor force participation rate
# (Civilian Labor Force / total civilian)
for i in group_list:
    # total civilian labor force estimate
    col1name = group_name[i] + 'CivLbr'
    # total civilians
    col2name =group_name[i] + 'Civ'
    # creates column name. 'CivLbr' stands for civilian labor force participation rate
    new_colname = group_name[i] + 'CivLbrRate'
    # divid labor force estimate by total civilians
    newcol = districts_df17B23002_grouped[col1name].astype('float64') / districts_df17B23002_grouped[col2name].astype('float64')
    districts_df17B23002_grouped[new_colname] = newcol
    # add calculated column to list of columns for summary dataframe
    summary_columns.append(new_colname)

In [25]:
# find count and percent of population outside the labor force
# out of labor force count
# (civilian male out of labor force + civilian female out of labor force)
for i in group_list:
    # total male civilian unemployment estimate
    col1name = i + '_009E'
    # total female civilian unemployment estimate
    col2name =i + '_048E'
    # creates column name. 'TotNonLbr' stands for total civilians not in labor force
    new_colname = group_name[i] + 'TotNonLbr'
    # add males to females. Census only includes these two categories
    newcol = districts_df17B23002_grouped[col1name].astype('int64') + districts_df17B23002_grouped[col2name].astype('int64')
    districts_df17B23002_grouped[new_colname] = newcol
    # add calculated column to list of columns for summary dataframe
    summary_columns.append(new_colname)
    
# find percent of civilians out of labor force
# (Civilian not in Labor Force / total civilian)
for i in group_list:
    # total civilian not in labor force estimate
    col1name = group_name[i] + 'TotNonLbr'
    # total civilians
    col2name =group_name[i] + 'Civ'
    # creates column name. 'CivLbr' stands for civilian labor force participation rate
    new_colname = group_name[i] + 'TotNonLbrRate'
    # divid labor force estimate by total civilians
    newcol = districts_df17B23002_grouped[col1name].astype('float64') / districts_df17B23002_grouped[col2name].astype('float64')
    districts_df17B23002_grouped[new_colname] = newcol
    # add calculated column to list of columns for summary dataframe
    summary_columns.append(new_colname)


In [26]:
# find civilian employed and unemployed counts

# employement count
# (civilian male employed + civilian female employed)
for i in group_list:
    # total male civilian employment estimate
    col1name = i + '_007E'
    # total female civilian employment estimate
    col2name =i + '_046E'
    # creates column name. 'TotEmply' stands for total civilians employed
    new_colname = group_name[i] + 'TotEmply'
    # add males to females. Census only includes these two categories
    newcol = districts_df17B23002_grouped[col1name].astype('int64') + districts_df17B23002_grouped[col2name].astype('int64')
    districts_df17B23002_grouped[new_colname] = newcol
    # add calculated column to list of columns for summary dataframe
    summary_columns.append(new_colname)

# unemployement count
# (civilian male unemployed + civilian female unemployed)
for i in group_list:
    # total male civilian unemployment estimate
    col1name = i + '_008E'
    # total female civilian unemployment estimate
    col2name =i + '_047E'
    # creates column name. 'TotEmply' stands for total civilians unemployed
    new_colname = group_name[i] + 'TotUnEmply'
    # add males to females. Census only includes these two categories
    newcol = districts_df17B23002_grouped[col1name].astype('int64') + districts_df17B23002_grouped[col2name].astype('int64')
    districts_df17B23002_grouped[new_colname] = newcol
    # add calculated column to list of columns for summary dataframe
    summary_columns.append(new_colname)
    


In [27]:
# find civilian employed and unemployed rates

# employement rate
# (total civilians employed / total civilian labor force)
for i in group_list:
    # total civilian employment estimate
    col1name = group_name[i] + 'TotEmply'
    # total civilian labor force
    col2name = group_name[i] + 'CivLbr'
    # creates column name. 'EmplyRate' stands for employment rate
    new_colname = group_name[i] + 'EmplyRate'
    # divide employment estimate by total labor force
    newcol = districts_df17B23002_grouped[col1name].astype('float64') / districts_df17B23002_grouped[col2name].astype('float64')
    districts_df17B23002_grouped[new_colname] = newcol
    # add calculated column to list of columns for summary dataframe
    summary_columns.append(new_colname)

# unemployement rate
# (total civilians unemployed / total civilian labor force)
for i in group_list:
    # total civilian unemployment estimate
    col1name = group_name[i] + 'TotUnEmply'
    # total civilian labor force
    col2name = group_name[i] + 'CivLbr'
    # creates column name. 'UnEmplyRate' stands for unemployment rate
    new_colname = group_name[i] + 'UnEmplyRate'
    # divide unemployment estimate by total labor force
    newcol = districts_df17B23002_grouped[col1name].astype('float64') / districts_df17B23002_grouped[col2name].astype('float64')
    districts_df17B23002_grouped[new_colname] = newcol
    # add calculated column to list of columns for summary dataframe
    summary_columns.append(new_colname)

In [28]:
summary_17B23002 = districts_df17B23002_grouped[summary_columns]
summary_17B23002.to_csv('output/summary_17B23002.csv')

### Preserved for reference
-- Notes from this test --
Good News:
    - The above call works
    - pd.DataFrame defaults to the desired row/column
To do:  
    - convert the existing labels (B14005_XXXX) into descriptive labels
    - find better way to get school district codes. There must be a better way than going through pdf maps
    - try Census wrapper library to see if it resolves district and state code issues
    - look into table S2301
    - figure out how we might use margin of error in analysis. If unneeded, maybe drop it
Big picture:
    - build loops to request all districts in each year
    - assemble district data into dataframe for each year
    
-- Notes on tables --
B23002A-B23002I give employment by age group for ethnic groups  
B14005 gives employment by enrollment for age groups  

    